# common

In [ ]:
//// test

open testing

## common

### (:>)

In [ ]:
prototype (~:>) r : forall t. t -> r

### to_any

In [ ]:
inl to_any forall t. (obj : t) : any =
    $'!obj '

instance (~:>) any = to_any

### (||>)

In [ ]:
//// test
///! fsharp
///! cuda
///! rust
///! typescript
///! python

(3i32, 2i32)
||> fun a b => a - b
|> _assert_eq 1

.py output (Cuda):
__assert_eq / actual: 1 / expected: 1

.rs output:
__assert_eq / actual: 1 / expected: 1

.ts output:
__assert_eq / actual: 1 / expected: 1

.py output:
__assert_eq / actual: 1 / expected: 1



.fsx output:
__assert_eq / actual: 1 / expected: 1


### flip

In [ ]:
inl flip fn a b =
    fn b a

In [ ]:
//// test
///! fsharp
///! cuda
///! rust
///! typescript
///! python

(1i32, 2i32)
||> flip pair
|> _assert_eq (2, 1)

.py output (Cuda):
__assert_eq / actual: (2, 1) / expected: (2, 1)

.rs output:
__assert_eq / actual: (2, 1) / expected: (2, 1)

.ts output:
__assert_eq / actual: 2,1 / expected: 2,1

.py output:
__assert_eq / actual: (2, 1) / expected: (2, 1)



.fsx output:
__assert_eq / actual: struct (2, 1) / expected: struct (2, 1)


### join_body

In [ ]:
inl join_body body acc x =
    if var_is x |> not
    then body acc x
    else
        inl acc = dyn acc
        join body acc x

In [ ]:
//// test

inl rec fold_list f s = function
    | Cons (x, x') => fold_list f (f s x) x'
    | Nil => s

In [ ]:
//// test
///! fsharp
///! cuda
///! rust
///! typescript
///! python
//// print_code=true

[ 5i32; 4; join 3; 2; 1 ]
|> fold_list (+) 0
|> _assert_eq 15

.py output (Cuda):
__assert_eq / actual: 15 / expected: 15

.rs output:
__assert_eq / actual: 15 / expected: 15

.ts output:
__assert_eq / actual: 15 / expected: 15

.py output:
__assert_eq / actual: 15 / expected: 15





.fsx:
let rec method1 () : int32 =
    3
and method2 (v0 : bool) : bool =
    v0
and closure0 (v0 : string) () : unit =
    let v1 : (string -> unit) = System.Console.WriteLine
    v1 v0
and method0 () : unit =
    let v0 : int32 = method1()
    let v1 : int32 = 9 + v0
    let v2 : int32 = v1 + 2
    let v3 : int32 = v2 + 1
    let v4 : bool = v3 = 15
    let v6 : bool =
        if v4 then
            true
        else
            method2(v4)
    let v7 : string = "__assert_eq"
    let v8 : string = $"{v7} / actual: %A{v3} / expected: %A{15}"
    let v11 : unit = ()
    let v12 : (unit -> unit) = closure0(v8)
    let v13 : unit = (fun () -> v12 (); v11) ()
    let v15 : bool = v6 = false
    if v15 then
        failwith<unit> v8
method0()


.rs:
#![allow(dead_code)]
#![allow(non_camel_case_types)]
#![allow(non_snake_case)]
#![allow(non_upper_case_globals)]
#![allow(unreachable_code)]
#![allow(unused_attributes)]
#![allow(unused_imports)]
#![allow(unused_macros)]
#![allow(unused_parens)

In [ ]:
//// test
///! fsharp
///! cuda
///! rust
///! typescript
///! python
//// print_code=true

[ 5i32; 4; join 3; 2; 1 ]
|> fold_list (join_body (+)) 0
|> _assert_eq 15

.py output (Cuda):
__assert_eq / actual: 15 / expected: 15

.rs output:
__assert_eq / actual: 15 / expected: 15

.ts output:
__assert_eq / actual: 15 / expected: 15

.py output:
__assert_eq / actual: 15 / expected: 15





.fsx:
let rec method1 () : int32 =
    3
and method2 (v0 : int32, v1 : int32) : int32 =
    let v2 : int32 = v1 + v0
    v2
and method3 (v0 : bool) : bool =
    v0
and closure0 (v0 : string) () : unit =
    let v1 : (string -> unit) = System.Console.WriteLine
    v1 v0
and method0 () : unit =
    let v0 : int32 = method1()
    let v1 : int32 = 9
    let v2 : int32 = method2(v0, v1)
    let v3 : int32 = v2 + 2
    let v4 : int32 = v3 + 1
    let v5 : bool = v4 = 15
    let v7 : bool =
        if v5 then
            true
        else
            method3(v5)
    let v8 : string = "__assert_eq"
    let v9 : string = $"{v8} / actual: %A{v4} / expected: %A{15}"
    let v12 : unit = ()
    let v13 : (unit -> unit) = closure0(v9)
    let v14 : unit = (fun () -> v13 (); v12) ()
    let v16 : bool = v7 = false
    if v16 then
        failwith<unit> v9
method0()


.rs:
#![allow(dead_code)]
#![allow(non_camel_case_types)]
#![allow(non_snake_case)]
#![allow(non_upper_case_globals)]
#![allow(unreach

### join_body_unit

In [ ]:
inl join_body_unit body d x =
    if var_is d |> not
    then body x
    else
        inl x = dyn x
        join body x

In [ ]:
//// test
///! fsharp
///! cuda
///! rust
///! typescript
///! python
//// print_code=true

[ 5i32; 4; join 3; 2; 1 ]
|> fold_list (fun acc n => join_body_unit ((+) acc) n n) 0
|> _assert_eq 15

.py output (Cuda):
__assert_eq / actual: 15 / expected: 15

.rs output:
__assert_eq / actual: 15 / expected: 15

.ts output:
__assert_eq / actual: 15 / expected: 15

.py output:
__assert_eq / actual: 15 / expected: 15





.fsx:
let rec method1 () : int32 =
    3
and method2 (v0 : int32) : int32 =
    let v1 : int32 = 9 + v0
    v1
and method3 (v0 : bool) : bool =
    v0
and closure0 (v0 : string) () : unit =
    let v1 : (string -> unit) = System.Console.WriteLine
    v1 v0
and method0 () : unit =
    let v0 : int32 = method1()
    let v1 : int32 = method2(v0)
    let v2 : int32 = v1 + 2
    let v3 : int32 = v2 + 1
    let v4 : bool = v3 = 15
    let v6 : bool =
        if v4 then
            true
        else
            method3(v4)
    let v7 : string = "__assert_eq"
    let v8 : string = $"{v7} / actual: %A{v3} / expected: %A{15}"
    let v11 : unit = ()
    let v12 : (unit -> unit) = closure0(v8)
    let v13 : unit = (fun () -> v12 (); v11) ()
    let v15 : bool = v6 = false
    if v15 then
        failwith<unit> v8
method0()


.rs:
#![allow(dead_code)]
#![allow(non_camel_case_types)]
#![allow(non_snake_case)]
#![allow(non_upper_case_globals)]
#![allow(unreachable_code)]
#![allow(unused_attributes)]

### retry_fn'

In [ ]:
inl retry_fn' retries fn =
    let rec loop retry =
        inl is_error, result =
            match fn () with
            | Ok x => false, x
            | Error x => true, x
        if not is_error || retry >= retries
        then result
        else
            trace Debug
                fun () => $'"common.retry_fn\' / loop"'
                fun () => { is_error retry = $'$"{!retry}/{!retries}"' : string; result }
            loop (retry + 1)
    loop 1

## fsharp

### upcast

In [ ]:
inl upcast forall t u. (x : t) : u =
    $'!x :> `u '

### downcast

In [ ]:
inl downcast forall t u. (x : t) : u =
    $'!x :?> `u '

### random

In [ ]:
nominal random = $'System.Random'

inl random () : random =
    $'`random ' ()

### random_next

In [ ]:
inl random_next (min : i32) (max : i32) (random : random) : i32 =
    $'!random.Next (!min, !max)'

### disposable

In [ ]:
nominal disposable t = $"backend_switch `({ Fsharp : $'System.IDisposable'; Python : $'object' })"

### dispose

In [ ]:
inl dispose (disposable : disposable _) : () =
    backend_switch {
        Fsharp = fun () => disposable |> $'_.Dispose()' : ()
        Python = fun () => $'!disposable.__exit__(None, None, None)' : ()
    }

### return

In [ ]:
inl return forall t. (x : t) : () =
    $'return !x '

inl return' forall t. (x : t) : t =
    $'return !x '

### retry_fn

In [ ]:
inl retry_fn forall t. retries (fn : () -> t) : option t =
    let rec loop retry =
        try
            fun () =>
                if retry < retries
                then fn () |> Some
                else None
            fun ex =>
                trace Warning
                    fun () => "common.retry_fn"
                    fun () => { retry ex }
                None
        |> function
            | Some x => x
            | None => loop (retry + 1)
    loop 0

In [ ]:
//// test
///! fsharp
////! cuda // v3 = $"retry: {v0} / ex: %A{v1} / {v2 ()}"
///! rust
///! typescript
///! python

inl retry_fn_test = mut 0i32
fun () =>
    retry_fn_test <- *retry_fn_test + 1
    *retry_fn_test
|> retry_fn 3i32
|> _assert_eq' (Some 1i32)

.rs output:
__assert_eq' / actual: US0_0(1) / expected: US0_0(1)

.ts output:
__assert_eq' / actual: US0_0 1 / expected: US0_0 1

.py output:
__assert_eq' / actual: US0_0 1 / expected: US0_0 1



.fsx output:
__assert_eq' / actual: US0_0 1 / expected: US0_0 1


In [ ]:
//// test
///! fsharp
////! cuda // v3 = $"retry: {v0} / ex: %A{v1} / {v2 ()}"
///! rust
///! typescript
///! python

inl retry_fn_test = mut 0i32
fun () =>
    if *retry_fn_test >= 2
    then *retry_fn_test
    else
        retry_fn_test <- *retry_fn_test + 1
        failwith "test"
|> retry_fn 3i32
|> _assert_eq' (Some 2i32)


.rs output:
00:00:00 warning #1 common.retry_fn / { retry = 0; ex = Exception {
    message: "test",
} }
00:00:00 warning #2 common.retry_fn / { retry = 1; ex = Exception {
    message: "test",
} }
__assert_eq' / actual: US0_0(2) / expected: US0_0(2)


.ts output:
00:00:00 warning #1 common.retry_fn / { retry = 0; ex = Error: test }
00:00:00 warning #2 common.retry_fn / { retry = 1; ex = Error: test }
__assert_eq' / actual: US0_0 2 / expected: US0_0 2


.py output:
00:00:00 warning #1 common.retry_fn / { retry = 0; ex = test }
00:00:00 warning #2 common.retry_fn / { retry = 1; ex = test }
__assert_eq' / actual: US0_0 2 / expected: US0_0 2




.fsx output:
00:00:00 warning #1 common.retry_fn / { retry = 0; ex = System.Exception: test
   at FSI_0031.closure1(Mut0 v0, Int32 v1, Unit unitVar2)
   at FSI_0031.method1(Mut0 v0, Int32 v1) }
00:00:00 warning #2 common.retry_fn / { retry = 1; ex = System.Exception: test
   at FSI_0031.closure1(Mut0 v0, Int32 v1, Unit unitVar2)
   at FSI_0031.method1(Mut0 v0, Int32 v1) }
__assert_eq' / actual: US0_0 2 / expected: US0_0 2


## common

### random'

In [ ]:
inl random' forall t. (min : t) (max : t) : t =
    run_target function
        | Rust (Contract) => fun () =>
            failwith "common.random' / target=Rust(Contract)"
        | Rust _ => fun () =>
            open rust.rust_operators
            !\\((min, max), $'"rand::Rng::gen_range(&mut rand::thread_rng(), $0..$1)"')
        | _ => fun () =>
            random () |> random_next (i32 min) (i32 max) |> convert

### new_disposable

In [ ]:
inl new_disposable (fn : () -> ()) : disposable _ =
    run_target function
        | Rust _ => fun () =>
            global "type Disposable (f : unit -> unit) = interface System.IDisposable with member _.Dispose () = f ()"
            inl fn = join fn
            $'new Disposable (fun () -> Fable.Core.RustInterop.emitRustExpr !fn "$0()" )'
        | Fsharp _ | TypeScript _ | Python _ => fun () =>
            inl fn = join fn
            $'{ new System.IDisposable with member _.Dispose () = !fn () }'
        | Cuda _ => fun () =>
            $'class Disposable:'
            $'    def __init__(self, fn):'
            $'        self.fn = fn'
            $'    def __exit__(self, exc_type, exc_value, traceback):'
            $'        self.fn()'
            $'        return False'
            $'Disposable(!fn)'
        | _ => fun () => null ()

In [ ]:
//// test
///! fsharp
///! cuda
///! rust
///! typescript
///! python

inl new_disposable_test = mut 0i32
new_disposable fun () => new_disposable_test <- *new_disposable_test + 1
|> fun x => x : disposable ()
|> dispose
*new_disposable_test |> _assert_eq 1

.py output (Cuda):
__assert_eq / actual: 1 / expected: 1

.rs output:
__assert_eq / actual: 1 / expected: 1

.ts output:
__assert_eq / actual: 1 / expected: 1

.py output:
__assert_eq / actual: 1 / expected: 1



.fsx output:
__assert_eq / actual: 1 / expected: 1


In [ ]:
//// test

inl new_disposable_test = mut 0i32
fun () =>
    new_disposable fun () => new_disposable_test <- *new_disposable_test + 1
    |> fun x => x : disposable ()
    |> use
    |> ignore
    |> return
|> async.new_task
|> async.await_task
|> async.run_synchronously
*new_disposable_test |> _assert_eq 1

__assert_eq / actual: 1 / expected: 1


In [ ]:
//// test

inl new_disposable_test = mut 0i32
fun () =>
    new_disposable fun () => new_disposable_test <- *new_disposable_test + 1
    |> fun x => x : disposable ()
    |> use
    |> ignore
    |> return
|> async.new_async
|> async.run_synchronously
*new_disposable_test |> _assert_eq 1

__assert_eq / actual: 1 / expected: 1


In [ ]:
//// test

inl new_disposable_test = mut 0i32
fun () =>
    new_disposable fun () => new_disposable_test <- *new_disposable_test + 1
    |> fun x => x : disposable ()
    |> ignore
    |> return
|> async.new_async
|> async.run_synchronously
*new_disposable_test |> _assert_eq 0

__assert_eq / actual: 0 / expected: 0


## main

In [ ]:
inl main () =
    init_trace_state None
    inl new_disposable x : _ () = new_disposable x
    $'let new_disposable x = !new_disposable x' : ()
    inl retry_fn (r : i32) (x : () -> _) : optionm'.option' () = retry_fn r x |> optionm'.box
    $'let retry_fn x = !retry_fn x' : ()
    inl memoize (fn : () -> ()) : () -> () = memoize fn
    $'let memoize x = !memoize x' : ()